# usual imports

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [13]:
def parse_amount(amount) :
    
    print("amount received : ", amount)
    
    if 'g' not in amount :
        raise ValueError('The amount string should contain mg.')
       
    elif '%' not in amount :
        if "μ" in amount :
            print("sososo : ", amount.replace("µ", ''))
            return str(float(re.sub('[mgμ]','',amount).strip())/1000.0)
        
        elif 'm' not in amount :
            #print("sososo : ", amount.replace("µ", ''))
            return str(float(re.sub('[mgμ]','',amount).strip())*1000.0)
        
        else :
            return re.sub('[mgμ]','',amount).strip()
    
    else :
        
        to_parse = ""
        
        #determine if % is before or after g
        if amount.index('%') < amount.index('g') :
            to_parse = amount[amount.index('%')+1:]
        
        else :
            to_parse = amount[:amount.index('g')]
            
        if 'μ' in to_parse :
            return str(float(re.sub('[mgμ]','',to_parse).strip())/1000.0)
        
        elif 'm' not in to_parse :
            return str(float(re.sub('[mgμ]','',to_parse).strip())*1000.0)
        
        else :
            return re.sub('[mgμ]','',to_parse).strip()
        

# load wanted elements names

In [3]:
elements_names = pd.read_excel('data/RDI.xlsx')['element'].values
elements_names

array(['alpha-linoleic acid', 'Biotin', 'Calcium', 'Carbohydrate',
       'Chloride', 'Choline', 'Chromium', 'Copper', 'Fat', 'Fiber',
       'Fluoride', 'Folate', 'Iodine', 'Iron', 'Linoleic acid',
       'Magnesium', 'Manganese', 'Molybdenum', 'Monousaturated fat',
       'Niacin', 'Pantothenic acid', 'Phosphorus', 'Polyunsaturated fat',
       'Potassium', 'Protein', 'Riboflavin', 'Saturated fat', 'Selenium',
       'Sodium', 'Sugar', 'Thiamin', 'Vitamin A', 'Vitamin B6',
       'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K',
       'Water', 'Zinc'], dtype=object)

# look up the wikipedia tables

In [4]:
html = urlopen("https://en.wikipedia.org/wiki/Avocado") 
res = BeautifulSoup(html.read(),"html5lib");

In [5]:
nutritional_facts = {}

table = res.find("table", {"class": "infobox nowrap"}) 
table_body = table.find('tbody')


rows = table_body.find_all('tr')
for row in rows:
    
    nut_element = ""
    
    valid_name = False
    found_name = ""
    
    #find elements names
    elem = row.find('th') 
    
    if elem is not None :
        name = elem.find('a') 
        
        if name is not None :
                        
            #check if the name contains something interesting
            matching = [c.lower() in name.text.lower() for c in elements_names]
            valid_name  = sum(matching) >= 1            
            
            if valid_name :
                matching_name = elements_names[matching.index(True)]
                found_name = name.text
                nut_element = matching_name
            
    #find element amount
    
    if valid_name :
        amount = row.find('td')
        print("found name : ", found_name)
        print("matching name : ", matching_name)

        if amount is not None :  
            
            #hard special case
            if found_name == "Vitamin A equiv." :
                whole_string = amount.text
                div_strings = [amount.find_all('div')[i].text for i in range(len(amount.find_all('div')))]
                print("div strings before sort: ", div_strings)
                div_strings = sorted(div_strings, key = len, reverse = True)
                print("div strings after sort : ", div_strings)
                
                print("whole string : ", whole_string)
                
                
                for div_string in div_strings :
                    whole_string = whole_string.replace(div_string, '')
                    
                print("after removal : ", whole_string)
                
                print("parsed : ",  parse_amount(whole_string), '\n')
                nutritional_facts[matching_name] = parse_amount(whole_string)

            elif 'div' not in str(amount) :
                
                print("text sent to parser : ", amount.text)
                print("parsed : ", parse_amount(amount.text), '\n')
                nutritional_facts[matching_name] = parse_amount(amount.text)

            else :              
                         
                print("raw text : ", amount.text)
                print("div text : ", amount.find_all('div')[0].text)
                
                #amount = parse_amount(amount.find_all('div')[0].text)
                amount = parse_amount(amount.text)
                print("parsed : ", amount, '\n')
                nutritional_facts[matching_name] = amount
            
nutritional_facts

found name :  Carbohydrates
matching name :  Carbohydrate
raw text :  8.53 g
div text :  8.53 g
amount received :  8.53 g
parsed :  8530.0 

found name :  Sugars
matching name :  Sugar
text sent to parser :  0.66 g
amount received :  0.66 g
parsed :  660.0 

amount received :  0.66 g
found name :  Dietary fiber
matching name :  Fiber
text sent to parser :  6.7 g
amount received :  6.7 g
parsed :  6700.0 

amount received :  6.7 g
found name :  Fat
matching name :  Fat
raw text :  14.66 g
div text :  14.66 g
amount received :  14.66 g
parsed :  14660.0 

found name :  Protein
matching name :  Protein
raw text :  2 g
div text :  2 g
amount received :  2 g
parsed :  2000.0 

found name :  Vitamin A equiv.
matching name :  Vitamin A
div strings before sort:  ['1%', '1%62 μg', '1%', '271 μg']
div strings after sort :  ['1%62 μg', '271 μg', '1%', '1%']
whole string :  1% 7 μg1%62 μg271 μg
after removal :   7 μg
amount received :   7 μg
sososo :   7 μg
parsed :  0.007 

amount received :   7 

ValueError: could not convert string to float: '7 µ'

In [14]:
parse_amount("7 µg")

amount received :  7 µg


ValueError: could not convert string to float: '7 µ'